## Import program dependencies

In [1]:
from imutils.video import VideoStream
import numpy as np
import imutils
import cv2 as cv

## Load the serialized black and white colorizer model and cluster center points

In [2]:
prototxt = '[YOUR WORKSPACE]\\colorization_deploy_v2.prototxt'
model = '[YOUR WORKSPACE]\\colorization_release_v2.caffemodel'
points = '[YOUR WORKSPACE]\\colorized\\pts_in_hull.npy'
video = '[YOUR WORKSPACE]\\[YOUR VIDEO.MP4]'
net = cv.dnn.readNetFromCaffe(prototxt, model)
pts = np.load(points)

## Determine number of frames and frame rate

In [3]:
vs = cv.VideoCapture(video)
number_of_frames = vs.get(cv.CAP_PROP_FRAME_COUNT)
frame_rate = vs.get(cv.CAP_PROP_FPS)
print(number_of_frames)

5466.0


In [4]:
print(frame_rate)

25.0


## Add the cluster centers as 1x1 convolutions to the model

In [5]:
class8 = net.getLayerId('class8_ab')
conv8 = net.getLayerId('conv8_313_rh')
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype('float32')]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype = 'float32')]

## Colorize and save each individual frame

In [6]:
count = 0
success = True
while success:
    success, frame = vs.read()
    if frame is None:
        break
    
    scaled = frame.astype('float32') / 255.0
    lab = cv.cvtColor(scaled, cv.COLOR_RGB2LAB)
    
    resized = cv.resize(lab, (224, 224))
    L = cv.split(resized)[0]
    L -= 50
    
    net.setInput(cv.dnn.blobFromImage(L))
    ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
    
    ab = cv.resize(ab, (frame.shape[1], frame.shape[0]))
    L = cv.split(lab)[0]
    colorized = np.concatenate((L[:, :, np.newaxis], ab), axis = 2)
    
    colorized = cv.cvtColor(colorized, cv.COLOR_LAB2BGR)
    colorized = np.clip(colorized, 0, 1)
    colorized = (255 * colorized).astype('uint8')

    cv.imwrite('[YOUR WORKSPACE]\\Frames\\Frame%d.jpg' % count, colorized)
    count += 1

vs.release()
cv.destroyAllWindows()

## Function for converting the individual frames back to video

In [12]:
def convert_frames_to_video(pathin, pathout):
    frame_array = []
    for lp in range(int(number_of_frames)):
        filename = pathin + 'Frame' + str(lp) + '.jpg'
        img = cv.imread(filename)
        height, width, layers = img.shape
        size = (width, height)
        frame_array.append(img)
    out = cv.VideoWriter(pathout, cv.VideoWriter_fourcc(*'MJPG'), frame_rate, size)
    
    for lp in range(len(frame_array)):
        out.write(frame_array[lp])
    out.release()

## Perform frame-to-video conversion

In [13]:
pathin = '[YOUR WORKSPACE]\\Frames\\'
pathout = '[YOUR WORKSPACE]\\[YOUR OUTPUT VIDEO.MP4]'
convert_frames_to_video(pathin, pathout)